In [2]:
import pandas as pd

df = pd.read_parquet('yellow_tripdata_2009-02.parquet')

df.head()


,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,DDS,2009-02-03 08:25:00,2009-02-03 08:33:39,1,1.60,-73.992768,40.758325,NaN,NaN,-73.994710,40.739723,CASH,6.9,0.0,NaN,0.0,0.0,6.9
1,VTS,2009-02-28 00:26:00,2009-02-28 00:40:00,5,3.05,0.000000,0.000000,NaN,NaN,0.000000,0.000000,CASH,10.5,0.5,NaN,0.0,0.0,11.0
2,DDS,2009-02-22 00:39:23,2009-02-22 00:45:52,1,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,5.7,0.5,NaN,0.0,0.0,6.2
3,VTS,2009-02-28 12:47:00,2009-02-28 12:54:00,1,1.42,-73.980458,40.748445,NaN,NaN,-73.996103,40.737095,CASH,6.1,0.0,NaN,0.0,0.0,6.1
4,DDS,2009-02-05 18:34:35,2009-02-05 18:43:26,1,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,6.9,1.0,NaN,0.0,0.0,7.9


In [3]:
print(df['store_and_forward'].value_counts(dropna=False))


store_and_forward
NaN    9009801
0.0    4268846
1.0     101475
Name: count, dtype: int64


In [4]:
print(df.isnull().sum())


vendor_name                     0
Trip_Pickup_DateTime            0
Trip_Dropoff_DateTime           0
Passenger_Count                 0
Trip_Distance                   0
Start_Lon                       0
Start_Lat                       0
Rate_Code                13380122
store_and_forward         9009801
End_Lon                         0
End_Lat                         0
Payment_Type                    0
Fare_Amt                        0
surcharge                       0
mta_tax                  13380122
Tip_Amt                         0
Tolls_Amt                       0
Total_Amt                       0
dtype: int64


In [5]:
len(df)


13380122

In [6]:
print(df['Payment_Type'].value_counts(dropna=False))


Payment_Type
CASH         5573418
Cash         4623482
Credit       2970754
CREDIT        161720
No Charge      42024
Dispute         8724
Name: count, dtype: int64


In [10]:
import pandas as pd
import geopandas as gpd
import pyarrow.parquet as pq
from shapely.geometry import Point

# Load NYC taxi zones shapefile
zones = gpd.read_file(r'F:\Data Engineering Projects\NYC Tax\taxi_zones\taxi_zones.shp')
zones = zones.to_crs(epsg=4326)

# Initialize final result DataFrame
final_df = pd.DataFrame()

# Load the Parquet file in chunks using PyArrow
parquet_file = pq.ParquetFile('yellow_tripdata_2009-02.parquet')

# Loop through batches
for i, batch in enumerate(parquet_file.iter_batches(batch_size=100_000)):
    print(f"Processing batch {i+1}")
    
    # Convert batch to pandas DataFrame
    df = batch.to_pandas()

    # Rename coordinates columns if needed
    df.rename(columns={
        'Start_Lon': 'start_lon',
        'Start_Lat': 'start_lat',
        'End_Lon': 'end_lon',
        'End_Lat': 'end_lat'
    }, inplace=True)

    # Drop missing coordinate rows
    df = df.dropna(subset=['start_lon', 'start_lat', 'end_lon', 'end_lat'])

    # ------------------------------
    # Start Locations → start_zone
    # ------------------------------
    start_points = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df['start_lon'], df['start_lat']),
        crs='EPSG:4326'
    )
    start_with_zone = gpd.sjoin(start_points, zones, how='left', predicate='within')
    df['start_zone'] = start_with_zone['zone'].values
    df['start_borough'] = start_with_zone['borough'].values

    # ------------------------------
    # End Locations → end_zone
    # ------------------------------
    end_points = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df['end_lon'], df['end_lat']),
        crs='EPSG:4326'
    )
    end_with_zone = gpd.sjoin(end_points, zones, how='left', predicate='within')
    df['end_zone'] = end_with_zone['zone'].values
    df['end_borough'] = end_with_zone['borough'].values

    # Drop geometry column to reduce memory
    df.drop(columns='geometry', inplace=True, errors='ignore')

    # Append to final DataFrame
    final_df = pd.concat([final_df, df], ignore_index=True)

    print(f"→ Batch {i+1} complete: {len(df)} records processed.")

# Save final enriched dataset to a new Parquet file (optional)
final_df.to_parquet('yellow_tripdata_2009-02_enriched.parquet', index=False)

print(f"\n✅ All done! Total enriched records: {len(final_df)}")


Processing batch 1
→ Batch 1 complete: 100000 records processed.
Processing batch 2
→ Batch 2 complete: 100000 records processed.
Processing batch 3
→ Batch 3 complete: 100000 records processed.
Processing batch 4
→ Batch 4 complete: 100000 records processed.
Processing batch 5
→ Batch 5 complete: 100000 records processed.
Processing batch 6
→ Batch 6 complete: 100000 records processed.
Processing batch 7
→ Batch 7 complete: 100000 records processed.
Processing batch 8
→ Batch 8 complete: 100000 records processed.
Processing batch 9
→ Batch 9 complete: 100000 records processed.
Processing batch 10
→ Batch 10 complete: 100000 records processed.
Processing batch 11
→ Batch 11 complete: 100000 records processed.
Processing batch 12
→ Batch 12 complete: 100000 records processed.
Processing batch 13
→ Batch 13 complete: 100000 records processed.
Processing batch 14
→ Batch 14 complete: 100000 records processed.
Processing batch 15
→ Batch 15 complete: 100000 records processed.
Processing ba

In [19]:
df.head(20)

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,start_lon,start_lat,Rate_Code,store_and_forward,end_lon,...,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt,start_zone,start_borough,end_zone,end_borough
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,NaN,NaN,-73.993803,...,8.9,0.5,NaN,0.00,0.0,9.40,Lower East Side,Manhattan,Brooklyn Heights,Brooklyn
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,NaN,NaN,-73.955850,...,12.1,0.5,NaN,2.00,0.0,14.60,Gramercy,Manhattan,Lenox Hill West,Manhattan
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,NaN,NaN,-73.869983,...,23.7,0.0,NaN,4.74,0.0,28.44,West Village,Manhattan,East Elmhurst,Queens
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,NaN,NaN,-73.996558,...,14.9,0.5,NaN,3.05,0.0,18.45,Upper West Side North,Manhattan,Greenwich Village North,Manhattan
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,NaN,NaN,-74.008378,...,3.7,0.0,NaN,0.00,0.0,3.70,Little Italy/NoLiTa,Manhattan,TriBeCa/Civic Center,Manhattan
5,DDS,2009-01-16 22:35:59,2009-01-16 22:43:35,2,1.20,-73.989806,40.735006,NaN,NaN,-73.985021,...,6.1,0.5,NaN,0.00,0.0,6.60,Gramercy,Manhattan,East Village,Manhattan
6,DDS,2009-01-21 08:55:57,2009-01-21 09:05:42,1,0.40,-73.984050,40.743544,NaN,NaN,-73.980260,...,5.7,0.0,NaN,1.00,0.0,6.70,Midtown South,Manhattan,Murray Hill,Manhattan
7,VTS,2009-01-04 04:31:00,2009-01-04 04:36:00,1,1.72,-73.992635,40.748362,NaN,NaN,-73.995585,...,6.1,0.5,NaN,0.00,0.0,6.60,Penn Station/Madison Sq West,Manhattan,Greenwich Village South,Manhattan
8,CMT,2009-01-05 16:29:02,2009-01-05 16:40:21,1,1.60,-73.969690,40.749244,NaN,NaN,-73.990413,...,8.7,0.0,NaN,1.30,0.0,10.00,UN/Turtle Bay South,Manhattan,Garment District,Manhattan
9,CMT,2009-01-05 18:53:13,2009-01-05 18:57:45,1,0.70,-73.955173,40.783044,NaN,NaN,-73.958598,...,5.9,0.0,NaN,0.00,0.0,5.90,Upper East Side North,Manhattan,Upper East Side North,Manhattan


In [20]:
df.to_parquet('yellow_tripdata_2009-02_enriched.parquet', index=False)
print("✅ Exported to yellow_tripdata_2009-02_enriched.parquet")

✅ Exported to yellow_tripdata_2009-02_enriched.parquet


In [4]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14092413 entries, 0 to 14092412
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   vendor_name            object 
 1   Trip_Pickup_DateTime   object 
 2   Trip_Dropoff_DateTime  object 
 3   Passenger_Count        int64  
 4   Trip_Distance          float64
 5   Start_Lon              float64
 6   Start_Lat              float64
 7   Rate_Code              float64
 8   store_and_forward      float64
 9   End_Lon                float64
 10  End_Lat                float64
 11  Payment_Type           object 
 12  Fare_Amt               float64
 13  surcharge              float64
 14  mta_tax                float64
 15  Tip_Amt                float64
 16  Tolls_Amt              float64
 17  Total_Amt              float64
dtypes: float64(13), int64(1), object(4)
memory usage: 1.9+ GB
None


In [6]:
count_non_null = df['Rate_Code'].count()
print("Number of non-null rate_code values in first 20 rows:", count_non_null)


Number of non-null rate_code values in first 20 rows: 0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14092413 entries, 0 to 14092412
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   vendor_name            object 
 1   Trip_Pickup_DateTime   object 
 2   Trip_Dropoff_DateTime  object 
 3   Passenger_Count        int64  
 4   Trip_Distance          float64
 5   Start_Lon              float64
 6   Start_Lat              float64
 7   Rate_Code              float64
 8   store_and_forward      float64
 9   End_Lon                float64
 10  End_Lat                float64
 11  Payment_Type           object 
 12  Fare_Amt               float64
 13  surcharge              float64
 14  mta_tax                float64
 15  Tip_Amt                float64
 16  Tolls_Amt              float64
 17  Total_Amt              float64
dtypes: float64(13), int64(1), object(4)
memory usage: 1.9+ GB
